# Excess Mortality in Europe and Why (part 2)
_Author: Justin Garza_

**Date:** See below  
  
**Description:**  
This notebook explores the causes of death and their changes over time.
> this is the part 2, part 1 can be seen here [Excess Mortality in Europe and Why]
(https://jgarza9788.github.io/Excess_Mortality_And_Vaccines_In_Europe/index.html)


**Content Warning:**    
If you find discussions of death or its underlying factors distressing, please proceed with caution or consider whether this content is right for you.  

In [1]:
from datetime import datetime
from IPython.display import display
from IPython.display import Markdown as MD
current_date = datetime.now().strftime('%Y-%m-%d')
version = datetime.now().strftime('%Y%m%d.%H%M')
display(MD(f"**Date:** {current_date}"))
display(MD(f"**version:** {version}"))

**Date:** 2025-01-06

**version:** 20250106.1953

## Background information

During late 2020 and much of 2021 many people took one or multiple covid vaccines.  
Width so many people taking the vaccines could we see a change in causes of death?

### the Covid-19 Vaccine
>"Messenger RNA (mRNA) vaccine. This type of vaccine gives your cells instructions for how to make the S protein found on the surface of the COVID-19 virus. After vaccination, your muscle cells begin making the S protein pieces and displaying them on cell surfaces. This causes your body to create antibodies." 
[www.mayoclinic.org](https://www.mayoclinic.org/diseases-conditions/coronavirus/in-depth/different-types-of-covid-19-vaccines/art-20506465#:~:text=Messenger%20RNA%20(mRNA)%20vaccine.,your%20body%20to%20create%20antibodies.)

### (Hypothesis) What are the medical issues we can we expect to see?
- **Cardiovascular diseases**: like heart disease, stroke, or coronary artery disease
    - due to the fact that this an injection and some might leak into the blood stream and cause issues
- **Liver disease**: chronic liver disease
    - the liver is the filter for the blood, so there is a possibility for the vaccine to effect the liver
- **Neurological disorders**: like Alzheimer's disease or Parkinson's disease    
    - an issue with the blood could inhibit how much oxygen gets to the brain
- **Cancers**
    - if the instructions leak into the DNA it could cause issues with the DNA and how cells reproduce.
- **Unknown Issues**
    - since this is a new type of vaccine there is a possibility of new issues appearing from it.


## SetUp
just like part 1

In [2]:
# this code to will import all the things i need for this notebook

import os
import re
import math

import numpy as np
import pandas as pd

import random
from collections import Counter


# for the notebook rendering 
from IPython.display import display, HTML, FileLink, Markdown
from IPython.display import FileLink as FL
from IPython.display import Markdown as MD


# Graphs and Charts
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import seaborn as sns
import plotly.express as px
# use to export plotly graphs
import plotly.io as pio 

#misc
from scipy.stats import spearmanr, kendalltau
import pycountry

# pandas Settings/Options
pd.set_option("display.max_rows", None) 
pd.set_option("display.max_columns", None)
pd.set_option('display.width', 9000)
pd.set_option('max_colwidth', 400)

# colormap 
heatmapCM = sns.color_palette('Spectral_r', as_cmap=True)


## directories 
DIR = os.getcwd()
print(f'{DIR=}')

DataDIR = os.path.join(DIR,'data')
OutDIR = os.path.join(DIR,'docs')

if not os.path.exists(DataDIR):
    print('***DATA FOLDER IS MISSING***')

if not os.path.exists(OutDIR):
    os.makedirs(OutDIR)

DIR='c:\\Users\\JGarza\\GitHub\\Excess_Mortality_And_Vaccines_In_Europe'


## Helping Functions


In [3]:
def df_column_uniquify(df):
    '''
    renames columns that are the same
    '''
    df_columns = df.columns
    new_columns = []
    for item in df_columns:
        counter = 0
        newitem = item
        while newitem in new_columns:
            counter += 1
            newitem = "{}_{}".format(item, counter)
        new_columns.append(newitem)
    df.columns = new_columns
    return df

In [4]:
def abbr_to_isoalpha3(abbr):
    """
    Convert a European country ISO Alpha-2 code to ISO Alpha-3 code.

    Parameters:
        abbreviation (str): ISO Alpha-2 country code (e.g., 'DE' for Germany).

    Returns:
        str: ISO Alpha-3 country code (e.g., 'DEU'), or None if not found.
    """
    try:
        country = pycountry.countries.get(alpha_2=abbr.upper())
        if country:
            return country.alpha_3
        else:
            return None
    except KeyError:
        return None

In [5]:
# this if for converting between the abbreviation andand the names of the countries

country_dict = {
    "BE": "Belgium",
    "BG": "Bulgaria",
    "CZ": "Czechia",
    "DK": "Denmark",
    "DE": "Germany",
    "EE": "Estonia",
    "IE": "Ireland",
    "EL": "Greece",
    "ES": "Spain",
    "FR": "France",
    "HR": "Croatia",
    "IT": "Italy",
    "CY": "Cyprus",
    "LV": "Latvia",
    "LT": "Lithuania",
    "LU": "Luxembourg",
    "HU": "Hungary",
    "MT": "Malta",
    "NL": "Netherlands",
    "AT": "Austria",
    "PL": "Poland",
    "PT": "Portugal",
    "RO": "Romania",
    "SI": "Slovenia",
    "SK": "Slovakia",
    "FI": "Finland",
    "SE": "Sweden",
    "IS": "Iceland",
    "LI": "Liechtenstein",
    "NO": "Norway",
    "CH": "Switzerland",
    "UK": "United Kingdom",
    "ME": "Montenegro",
    "GE": "Georgia",
    "AL": "Albania",
    "RS": "Serbia",
    "AD": "Andorra",
    "AM": "Armenia",
}

def abbr_to_name(abbreviation):
    return country_dict.get(abbreviation.upper(), "???")

def name_to_abbr(name):
    try:
        reverse_dict = {v: k for k, v in country_dict.items()}
        return reverse_dict.get(name.title(), "???")
    except:
        return "???"

# testing
print( abbr_to_name("BE") )  # Output: 'Belgium'
print( name_to_abbr("Belgium") )  # Output: 'BE'

Belgium
BE


In [6]:

# define now... put real values in later 
ddr2023 = None
ddr2024 = None
vdr2023 = None
vdravg  = None

def return_ddr2023_rank(abbr):
    '''
    the higher the value the higher the death 
    '''

    if ddr2023 is None:
        print('***DDR2023 DATA IS NOT LOADED***')
        return None
    
    for i, row in ddr2023.iterrows():
        if abbr == row['abbr']:
            return 1.0 - ( i/len(ddr2023) )

def return_vdr2023_rank(abbr):
    '''
    the higher the value the more vaccines
    '''

    if vdr2023 is None:
        print('***vdr2023 DATA IS NOT LOADED***')
        return None
    
    for i, row in vdr2023.iterrows():
        if abbr == row['abbr']:
            return 1.0 - ( i/len(ddr2023) )
        

def return_vdravg_rank(abbr):
    '''
    the higher the value the more vaccines
    '''

    if vdravg is None:
        print('***vdravg DATA IS NOT LOADED***')
        return None
    
    for i, row in vdravg.iterrows():
        if abbr == row['abbr']:
            return 1.0 - ( i/len(ddr2023) )


In [7]:
print(return_ddr2023_rank('MT'))
print(return_ddr2023_rank('CY'))

print(return_vdr2023_rank('MT'))
print(return_vdr2023_rank('CY'))

print(return_vdravg_rank('MT'))
print(return_vdravg_rank('CY'))

***DDR2023 DATA IS NOT LOADED***
None
***DDR2023 DATA IS NOT LOADED***
None
***vdr2023 DATA IS NOT LOADED***
None
***vdr2023 DATA IS NOT LOADED***
None
***vdravg DATA IS NOT LOADED***
None
***vdravg DATA IS NOT LOADED***
None


## import data from part 1
### Variables
- `ddr2023`: Death data ranked by deaths in 2023.
- `ddr2024`: Death data ranked by deaths in 2024.
- `vdr2023`: Vaccination data ranked by the cumulative sum for 2023.
- `vdravg`: Vaccination data ranked by the average cumulative sum for 2020, 2021, 2022, and 2023.


In [8]:
ddr2023 = pd.read_csv(os.path.join(OutDIR,'ddr2023.csv'))
ddr2024 = pd.read_csv(os.path.join(OutDIR,'ddr2024.csv'))
vdr2023 = pd.read_csv(os.path.join(OutDIR,'vdr2023.csv'))
vdravg = pd.read_csv(os.path.join(OutDIR,'vdravg.csv'))

In [9]:
display(ddr2023.head(5))
display(ddr2024.head(5))
display(vdr2023.head(5))
display(vdravg.head(5))

,abbr,name,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,MT,Malta,1.004157,0.961827,1.033935,1.067663,1.071152,1.184449,1.203738,1.226999,1.169429,1.195514
1,CY,Cyprus,1.016240,0.943163,1.040284,1.003756,1.076467,1.129533,1.249761,1.247856,1.157833,1.095931
2,FI,Finland,0.983845,1.009937,1.006529,1.023471,1.011123,1.038642,1.083859,1.163166,1.150723,1.071371
3,IS,Iceland,0.975717,1.026691,0.998060,1.006559,1.012375,1.029265,1.045927,1.203398,1.147478,1.163664
4,NL,Netherlands,0.990752,0.999534,1.009892,1.031217,1.021614,1.133312,1.147735,1.140822,1.138502,1.149227


,abbr,name,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,MT,Malta,1.004157,0.961827,1.033935,1.067663,1.071152,1.184449,1.203738,1.226999,1.169429,1.195514
1,IS,Iceland,0.975717,1.026691,0.998060,1.006559,1.012375,1.029265,1.045927,1.203398,1.147478,1.163664
2,NL,Netherlands,0.990752,0.999534,1.009892,1.031217,1.021614,1.133312,1.147735,1.140822,1.138502,1.149227
3,LI,Liechtenstein,0.984876,1.050506,0.964909,1.062178,1.011598,1.240639,1.038833,1.085522,1.027161,1.127824
4,IE,Ireland,0.994335,1.004840,1.000934,1.020105,1.008454,1.028312,1.105698,1.135294,1.132809,1.113159


,abbr,name,2020,2021,2022,2023
0,FI,Finland,0.003119,123.689548,324.096693,272.896203
1,PT,Portugal,0.008074,94.765752,242.524514,206.590389
2,IT,Italy,0.004416,87.617659,233.812064,185.254382
3,SE,Sweden,0.002223,80.134686,207.824977,169.837101
4,FR,France,0.000154,85.965138,222.847236,123.812258


,abbr,name,avg2020_2023
0,FI,Finland,180.171391
1,PT,Portugal,135.972182
2,IT,Italy,126.672130
3,SE,Sweden,114.449747
4,FR,France,108.156197


In [10]:
print(return_ddr2023_rank('MT'))
print(return_ddr2023_rank('CY'))

print(return_vdr2023_rank('MT'))
print(return_vdr2023_rank('CY'))

print(return_vdravg_rank('MT'))
print(return_vdravg_rank('CY'))

1.0
0.9696969696969697
0.6363636363636364
0.48484848484848486
0.7878787878787878
0.5151515151515151


## Cause of Death Data
In this section, we import and clean the cause of death data to prepare it for analysis.

### Source
- [Eurostat Database](https://ec.europa.eu/eurostat/databrowser/view/hlth_cd_aro__custom_14850625/default/table?lang=en)

### Downloading Instructions
1. Visit the [source link](https://ec.europa.eu/eurostat/databrowser/view/hlth_cd_aro__custom_14850625/default/table?lang=en).
2. Click "Download" and select the spreadsheet format.
3. Confirm the download to obtain the data.

### Description
This is the country, Cause of Death, and the count of each death between 2015 and 2023*.  
* note: much of 2023 is not reported

### variables 
- `cod` = cause of death data 
- `cod20dd102023` = list of the top 20 cause of deaths with countries in the top 10 death data ranked by deaths in 2023


In [11]:
# getting the data
cod = pd.read_excel(os.path.join(DataDIR,"hlth_cd_aro__custom_14850625_page_spreadsheet.xlsx"),sheet_name="Sheet 1")

# remove the headers
cod = cod.iloc[9::]

# drop the bad columns
for c in cod.columns:
    if pd.isnull(cod.at[9,c]):
        cod = cod.drop(columns=[c])

# rename time columns
for c in cod.columns:
    name = cod.at[9,c]
    cod = cod.rename(columns={c: name})

# make the duplicate column names unique
cod = df_column_uniquify(cod)

# # rename the first two columns
cod = cod.rename(columns={'TIME': 'name'})
cod = cod.rename(columns={'TIME_1':'cod'})

# drop, replace, reset index,
cod = cod.drop([9,10])
cod = cod.replace(to_replace=':', value=None)
cod = cod.reset_index(drop=True)

display(cod.head(10))


c:\Users\JGarza\pythons\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,name,cod,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,European Union - 27 countries (from 2020),Total,4618660,4536667,4660352,4693334,4660270,5189007,5298536,None,None
1,European Union - 27 countries (from 2020),Certain infectious and parasitic diseases (A00-B99),81735,74272,77707,77359,77128,73199,75560,None,None
2,European Union - 27 countries (from 2020),Tuberculosis,4098,3809,3648,3597,3382,3041,2943,None,None
3,European Union - 27 countries (from 2020),Viral hepatitis and sequelae of viral hepatitis,6956,6356,5857,4861,4396,4150,3587,None,None
4,European Union - 27 countries (from 2020),Chronic viral hepatitis B and C,1127,4814,1020,4273,3830,3529,3028,None,None
5,European Union - 27 countries (from 2020),Human immunodeficiency virus [HIV] disease,3234,2737,2438,2366,2173,2027,1891,None,None
6,European Union - 27 countries (from 2020),Other infectious and parasitic diseases (remainder of A00-B99),67284,61174,65666,66119,66788,64033,67197,None,None
7,European Union - 27 countries (from 2020),Neoplasms,1200230,1213401,1212135,1216939,1218306,1215992,1193515,None,None
8,European Union - 27 countries (from 2020),"Malignant neoplasm of lip, oral cavity, pharynx",27968,28456,28285,28573,28094,27845,27571,None,None
9,European Union - 27 countries (from 2020),"Malignant neoplasm of colon, rectosigmoid junction, rectum, anus and anal canal",137921,139889,137718,137468,137428,135701,132364,None,None


In [12]:
# lets get the abbr

cod['abbr'] = cod['name'].apply(name_to_abbr)

cod.loc[cod.name.str.contains('28 countries') == True,'abbr'] = 'Eur28'
cod.loc[cod.name.str.contains('27 countries') == True,'abbr'] = 'Eur27'

# France is France
cod.loc[cod.name.str.contains('Metropolitan France') == True,'abbr'] = 'Fr'

# Turkey is Tr
cod.loc[cod.name.str.contains('Türkiye') == True,'abbr'] = 'Tr'

# lets exclude the Eur28 and Eur27
cod = cod[~cod['abbr'].isin(['Eur28', 'Eur27'])]

display(cod.tail(250).head(10))
# display(cod.head(10))


,name,cod,2015,2016,2017,2018,2019,2020,2021,2022,2023,abbr
1975,Norway,Intentional self-harm,598,627,603,681,666,650,657,None,None,NO
1976,Norway,Accidental poisoning by and exposure to noxious substances,324,348,298,331,323,376,322,None,None,NO
1977,Norway,Assault,25,36,30,24,31,36,31,None,None,NO
1978,Norway,Event of undetermined intent,7,5,5,6,8,12,12,None,None,NO
1979,Norway,Other external causes of morbidity and mortality (remainder of V01-Y89),2,1,0,2,5,7,6,None,None,NO
1980,Switzerland,Total,68279,65533,67431,67621,68424,76853,71848,75199,None,CH
1981,Switzerland,Certain infectious and parasitic diseases (A00-B99),849,765,825,823,826,851,912,1001,None,CH
1982,Switzerland,Tuberculosis,17,25,19,25,9,13,15,12,None,CH
1983,Switzerland,Viral hepatitis and sequelae of viral hepatitis,81,61,50,13,17,52,42,33,None,CH
1984,Switzerland,Chronic viral hepatitis B and C,None,None,0,9,11,6,7,11,None,CH


In [13]:
# reformat the year

cod =  pd.melt(cod, id_vars=['name', 'abbr','cod'], var_name='year', value_name='deaths')

# we don't need cod total
cod = cod[cod.cod!= 'Total']

cod.year = pd.to_numeric(cod.year)
cod.deaths = pd.to_numeric(cod.deaths)
display(cod.head(10))

,name,abbr,cod,year,deaths
1,Belgium,BE,Certain infectious and parasitic diseases (A00-B99),2015,2361.0
2,Belgium,BE,Tuberculosis,2015,57.0
3,Belgium,BE,Viral hepatitis and sequelae of viral hepatitis,2015,72.0
4,Belgium,BE,Chronic viral hepatitis B and C,2015,NaN
5,Belgium,BE,Human immunodeficiency virus [HIV] disease,2015,32.0
6,Belgium,BE,Other infectious and parasitic diseases (remainder of A00-B99),2015,2200.0
7,Belgium,BE,Neoplasms,2015,28845.0
8,Belgium,BE,"Malignant neoplasm of lip, oral cavity, pharynx",2015,588.0
9,Belgium,BE,"Malignant neoplasm of colon, rectosigmoid junction, rectum, anus and anal canal",2015,2874.0
10,Belgium,BE,"Malignant neoplasm of trachea, bronchus and lung",2015,6318.0


In [14]:
# filter out bad values

## old way of doing it
# cod = cod.replace([np.inf, -np.inf], np.nan).dropna()

cod = cod.map(lambda x: np.nan if x in [np.inf, -np.inf] else x).dropna()

In [15]:
# here we normalize the cause of death data

cod['deaths_norm'] = None
cod.reset_index(drop=True)

# print(cod.columns)

## baseline years are 2015,2016,2017 before the pandemic 
blyears = cod[cod.year.isin([2015,2016,2017])]
grouped = blyears.groupby(['name','abbr','cod'])

temp = grouped['deaths'].agg(
    baseline='mean',
).reset_index()

for _, row in temp.iterrows():
    try:
        cod.loc[(cod.abbr == row['abbr']) & (cod.cod == row['cod']) & (cod.name == row['name']), 'deaths_norm'] = cod[(cod.abbr == row['abbr']) & (cod.cod == row['cod']) & (cod.name == row['name'])].deaths / row['baseline']
    except Exception as e:
        # can't caluate them all 
        print(f"Error: {e} | {row=}")

cod.deaths_norm = pd.to_numeric(cod.deaths_norm)


In [16]:
# integrate Part 1 data

cod['ddr2023_rank'] = cod.abbr.apply(return_ddr2023_rank)
cod['vdr2023_rank'] = cod.abbr.apply(return_vdr2023_rank)
cod['vdravg_rank'] = cod.abbr.apply(return_vdravg_rank)



### viewing some data


In [17]:
display(cod.head(10))

,name,abbr,cod,year,deaths,deaths_norm,ddr2023_rank,vdr2023_rank,vdravg_rank
1,Belgium,BE,Certain infectious and parasitic diseases (A00-B99),2015,2361.0,1.034921,0.333333,0.666667,0.69697
2,Belgium,BE,Tuberculosis,2015,57.0,1.315385,0.333333,0.666667,0.69697
3,Belgium,BE,Viral hepatitis and sequelae of viral hepatitis,2015,72.0,1.102041,0.333333,0.666667,0.69697
5,Belgium,BE,Human immunodeficiency virus [HIV] disease,2015,32.0,0.914286,0.333333,0.666667,0.69697
6,Belgium,BE,Other infectious and parasitic diseases (remainder of A00-B99),2015,2200.0,1.033835,0.333333,0.666667,0.69697
7,Belgium,BE,Neoplasms,2015,28845.0,1.006525,0.333333,0.666667,0.69697
8,Belgium,BE,"Malignant neoplasm of lip, oral cavity, pharynx",2015,588.0,1.002843,0.333333,0.666667,0.69697
9,Belgium,BE,"Malignant neoplasm of colon, rectosigmoid junction, rectum, anus and anal canal",2015,2874.0,1.009129,0.333333,0.666667,0.69697
10,Belgium,BE,"Malignant neoplasm of trachea, bronchus and lung",2015,6318.0,1.025372,0.333333,0.666667,0.69697
11,Belgium,BE,Malignant neoplasm of other parts of uterus,2015,364.0,0.934932,0.333333,0.666667,0.69697


#### HeatMaps
> note: 2023 data is skewed due to lack of reporting

In [26]:
year_columns = [2015,2016,2017,2018,2019,2020,2021,2022,2023]

In [42]:
temp = cod[cod.abbr.isin(ddr2024.head(10).abbr)].replace([np.inf, -np.inf], np.nan).dropna()

temp = pd.pivot_table(
        data =  temp,
        values='deaths_norm',
        index = ['abbr','name','cod','ddr2023_rank','vdr2023_rank','vdravg_rank'],
        columns='year',
        aggfunc='mean'
    )
temp.columns.name = 'index' 
temp = temp.reset_index()

abbrs = ddr2024.head(10).abbr.to_list()

cod_counter = Counter()

for a in abbrs:
    temp2 = temp[temp.abbr == a]
    temp2 = temp2.sort_values(by=2020,ascending=False)
    temp2 = temp2.sort_values(by=2021,ascending=False)
    temp2 = temp2.sort_values(by=2022,ascending=False)
    temp2 = temp2.sort_values(by=2023,ascending=False)
    display(MD(f'### {a} Causes of Death'))
    display(temp2.head(20).style.background_gradient(cmap=heatmapCM,axis=1,subset=year_columns)) 

    cod_counter.update(temp2['cod'].head(20).tolist())


html_table = f"""
<h>Top 20 Causes of Death and the counts they appear in the heatmaps above (see above)</h>
<table>
    <tr>
        <th>Item</th>
        <th>Count</th>
        <th>bar</th>
    </tr>
    {''.join(f'<tr><td>{i[0]}</td><td>{i[1]}</td><td>{"#" * i[1]}</td></tr>' for i in cod_counter.most_common()[0:20])}
</table>
"""

display(HTML(html_table))

### MT Causes of Death

index,abbr,name,cod,ddr2023_rank,vdr2023_rank,vdravg_rank,2015,2016,2017,2018,2019,2020,2021,2022,2023
411,MT,Malta,Ill-defined and unknown causes of mortality,1.000000,0.636364,0.787879,1.875000,0.375000,0.750000,3.750000,5.625000,4.125000,3.375000,nan,nan
431,MT,Malta,Other external causes of morbidity and mortality (remainder of V01-Y89),1.000000,0.636364,0.787879,1.000000,1.000000,1.000000,3.000000,2.000000,5.000000,3.000000,nan,nan
433,MT,Malta,Other infectious and parasitic diseases (remainder of A00-B99),1.000000,0.636364,0.787879,0.708333,1.041667,1.250000,2.000000,1.833333,2.708333,2.666667,nan,nan
430,MT,Malta,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",1.000000,0.636364,0.787879,0.825000,0.900000,1.275000,0.975000,1.575000,1.500000,2.175000,nan,nan
396,MT,Malta,Certain infectious and parasitic diseases (A00-B99),1.000000,0.636364,0.787879,0.750000,1.173913,1.076087,1.630435,1.597826,2.184783,2.152174,nan,nan
401,MT,Malta,Dementia,1.000000,0.636364,0.787879,0.846316,0.966316,1.187368,1.610526,1.446316,2.040000,2.103158,nan,nan
438,MT,Malta,Other mental and behavioural disorders (remainder of F00-F99),1.000000,0.636364,0.787879,2.000000,0.500000,0.500000,1.500000,1.500000,0.500000,2.000000,nan,nan
408,MT,Malta,Falls,1.000000,0.636364,0.787879,0.788321,1.226277,0.985401,0.919708,1.138686,1.160584,1.686131,nan,nan
415,MT,Malta,Leukaemia,1.000000,0.636364,0.787879,0.929577,0.676056,1.394366,1.140845,1.014085,1.521127,1.521127,nan,nan
426,MT,Malta,Other diseases of the genitourinary system (remainder of N00-N99),1.000000,0.636364,0.787879,0.992958,0.760563,1.246479,0.802817,0.950704,0.908451,1.521127,nan,nan


### IS Causes of Death

index,abbr,name,cod,ddr2023_rank,vdr2023_rank,vdravg_rank,2015,2016,2017,2018,2019,2020,2021,2022,2023
248,IS,Iceland,Ill-defined and unknown causes of mortality,0.909091,0.181818,0.393939,nan,nan,1.000000,3.000000,6.000000,11.000000,8.000000,nan,nan
263,IS,Iceland,Other diseases of the genitourinary system (remainder of N00-N99),0.909091,0.181818,0.393939,0.913043,1.434783,0.652174,1.304348,1.173913,1.695652,2.347826,nan,nan
275,IS,Iceland,Other mental and behavioural disorders (remainder of F00-F99),0.909091,0.181818,0.393939,0.857143,0.214286,1.928571,0.857143,1.285714,1.071429,2.142857,nan,nan
259,IS,Iceland,Non-malignant neoplasms (benign and uncertain),0.909091,0.181818,0.393939,1.285714,1.142857,0.571429,1.142857,1.428571,1.285714,2.000000,nan,nan
268,IS,Iceland,Other external causes of morbidity and mortality (remainder of V01-Y89),0.909091,0.181818,0.393939,0.857143,0.428571,1.714286,3.000000,3.857143,2.571429,1.714286,nan,nan
278,IS,Iceland,"Rheumatoid arthritis and arthrosis (M05-M06,M15-M19)",0.909091,0.181818,0.393939,0.750000,1.500000,0.750000,3.750000,5.250000,1.500000,1.500000,nan,nan
239,IS,Iceland,Diabetes mellitus,0.909091,0.181818,0.393939,0.775281,1.044944,1.179775,1.247191,1.011236,1.112360,1.483146,nan,nan
240,IS,Iceland,Diseases of kidney and ureter,0.909091,0.181818,0.393939,1.229508,0.983607,0.786885,1.377049,1.229508,1.426230,1.475410,nan,nan
228,IS,Iceland,Accidental poisoning by and exposure to noxious substances,0.909091,0.181818,0.393939,0.685714,0.814286,1.500000,2.271429,1.414286,1.071429,1.457143,nan,nan
238,IS,Iceland,Dementia,0.909091,0.181818,0.393939,1.047619,0.920635,1.031746,1.285714,1.412698,1.444444,1.444444,nan,nan


### NL Causes of Death

...

### PT Causes of Death

index,abbr,name,cod,ddr2023_rank,vdr2023_rank,vdravg_rank,2015,2016,2017,2018,2019,2020,2021,2022,2023
519,PT,Portugal,"Drug dependence, toxicomania (F11-F16, F18-F19)",0.696970,0.969697,0.969697,1.434783,0.391304,1.173913,1.043478,1.565217,1.695652,2.347826,nan,nan
552,PT,Portugal,Other mental and behavioural disorders (remainder of F00-F99),0.696970,0.969697,0.969697,0.917197,1.136943,0.945860,1.337580,2.025478,2.149682,2.207006,nan,nan
503,PT,Portugal,Accidental poisoning by and exposure to noxious substances,0.696970,0.969697,0.969697,0.864629,0.917031,1.218341,1.401747,1.414847,1.296943,2.148472,nan,nan
525,PT,Portugal,Ill-defined and unknown causes of mortality,0.696970,0.969697,0.969697,1.019554,0.976008,1.004439,1.153791,1.454295,1.755518,1.835053,nan,nan
514,PT,Portugal,Dementia,0.696970,0.969697,0.969697,0.887734,1.004329,1.107937,1.337374,1.544589,1.751804,1.662915,nan,nan
537,PT,Portugal,"Other accidents (W20-W64, W75-X39, X50-X59, Y86)",0.696970,0.969697,0.969697,0.769384,1.017325,1.213292,1.176654,0.605794,1.011360,1.583073,nan,nan
547,PT,Portugal,Other infectious and parasitic diseases (remainder of A00-B99),0.696970,0.969697,0.969697,0.930394,0.995789,1.073817,1.050037,0.995046,1.028486,1.405251,nan,nan
544,PT,Portugal,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",0.696970,0.969697,0.969697,1.068902,0.974587,0.956510,0.989521,0.877129,1.107414,1.393503,nan,nan
504,PT,Portugal,"Accidents (V01-X59, Y85, Y86)",0.696970,0.969697,0.969697,0.892690,0.983984,1.123327,1.093811,0.892690,1.057087,1.302483,nan,nan
545,PT,Portugal,Other external causes of morbidity and mortality (remainder of V01-Y89),0.696970,0.969697,0.969697,1.018310,1.073239,0.908451,0.954930,1.212676,1.276056,1.297183,nan,nan


### FI Causes of Death

index,abbr,name,cod,ddr2023_rank,vdr2023_rank,vdravg_rank,2015,2016,2017,2018,2019,2020,2021,2022,2023
134,FI,Finland,Ill-defined and unknown causes of mortality,0.939394,1.000000,1.000000,1.464883,0.675585,0.859532,0.735786,0.682274,0.789298,1.535117,2.046823,nan
128,FI,Finland,"Drug dependence, toxicomania (F11-F16, F18-F19)",0.939394,1.000000,1.000000,0.579545,1.431818,0.988636,1.602273,1.363636,1.738636,1.840909,1.738636,nan
147,FI,Finland,Other diseases of the circulatory system (remainder of I00-I99),0.939394,1.000000,1.000000,0.949102,1.001744,1.049153,1.095639,1.214777,1.300975,1.339764,1.535557,nan
153,FI,Finland,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",0.939394,1.000000,1.000000,0.856354,1.088398,1.055249,1.110497,1.149171,1.215470,1.226519,1.497238,nan
161,FI,Finland,Other mental and behavioural disorders (remainder of F00-F99),0.939394,1.000000,1.000000,0.987578,1.024845,0.987578,1.229814,1.217391,1.198758,1.409938,1.490683,nan
123,FI,Finland,Dementia,0.939394,1.000000,1.000000,0.998815,1.005927,0.995258,1.101439,1.083658,1.144623,1.305165,1.429128,nan
151,FI,Finland,Other diseases of the nervous system and the sense organs (remainder of G00-H95),0.939394,1.000000,1.000000,0.967711,0.992403,1.039886,1.081671,1.171890,1.238367,1.246914,1.360874,nan
124,FI,Finland,Diabetes mellitus,0.939394,1.000000,1.000000,0.910006,1.021397,1.068597,1.017621,1.125236,1.261171,1.270610,1.346130,nan
127,FI,Finland,Diseases of the nervous system and the sense organs (G00-H95),0.939394,1.000000,1.000000,0.938935,1.013457,1.047608,1.116024,1.133439,1.194730,1.254099,1.342644,nan
150,FI,Finland,Other diseases of the musculoskeletal system and connective tissue (remainder of M00-M99),0.939394,1.000000,1.000000,0.974138,1.043103,0.982759,1.146552,0.939655,1.051724,1.137931,1.241379,nan


Item,Count,bar
Ill-defined and unknown causes of mortality,9,#########
"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",9,#########
Other mental and behavioural disorders (remainder of F00-F99),9,#########
Dementia,8,########
Other diseases of the nervous system and the sense organs (remainder of G00-H95),8,########
Other diseases of the genitourinary system (remainder of N00-N99),7,#######
Accidental poisoning by and exposure to noxious substances,7,#######
Other diseases of the circulatory system (remainder of I00-I99),6,######
Chronic liver disease,6,######
Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism,6,######


In [19]:
temp = cod.replace([np.inf, -np.inf], np.nan).dropna()

temp = pd.pivot_table(
        data =  temp,
        values='deaths_norm',
        index = ['abbr','name','cod','ddr2023_rank','vdr2023_rank','vdravg_rank'],
        columns='year',
        aggfunc='mean'
    )
temp.columns.name = 'index' 
temp = temp.reset_index()
temp = temp.sort_values(by=2022,ascending=False)

display(MD('### Top 20 Causes of Death in 2022'))
display(temp.head(20).style.background_gradient(cmap=heatmapCM,axis=1,subset=year_columns))

display(MD('### Causes of Death in countries with a high 2023 death ranking'))
temp2 = temp[temp.abbr.isin(ddr2023.abbr.head(10))]
temp2 = temp2[temp2[2022] >= 1.5]
temp2 = temp2.sort_values(by='ddr2023_rank',ascending=False)
display(temp2.style.background_gradient(cmap=heatmapCM,axis=1,subset=year_columns))

### Top 20 Causes of Death in 2022

index,abbr,name,cod,ddr2023_rank,vdr2023_rank,vdravg_rank,2015,2016,2017,2018,2019,2020,2021,2022,2023
824,IE,Ireland,"Drug dependence, toxicomania (F11-F16, F18-F19)",0.848485,0.757576,0.727273,2.000000,nan,0.000000,2.000000,0.000000,10.000000,8.000000,32.000000,nan
709,HR,Croatia,Event of undetermined intent,0.181818,0.272727,0.212121,0.387097,0.483871,2.129032,2.370968,4.403226,5.225806,7.790323,11.225806,nan
1615,SI,Slovenia,"Drug dependence, toxicomania (F11-F16, F18-F19)",0.666667,0.303030,0.272727,nan,0.857143,1.142857,2.857143,3.428571,7.142857,8.000000,9.428571,nan
986,LI,Liechtenstein,Dementia,0.424242,0.454545,0.424242,nan,1.500000,0.500000,1.500000,0.500000,8.500000,6.000000,7.500000,9.000000
223,CY,Cyprus,Other mental and behavioural disorders (remainder of F00-F99),0.969697,0.484848,0.515152,0.818182,1.363636,0.818182,2.454545,2.181818,4.636364,6.272727,7.363636,nan
1420,PL,Poland,Other mental and behavioural disorders (remainder of F00-F99),0.454545,0.818182,0.818182,0.637500,1.368750,0.993750,1.181250,1.275000,1.593750,8.212500,6.356250,nan
944,IT,Italy,Ill-defined and unknown causes of mortality,0.363636,0.939394,0.939394,0.917200,0.989683,1.093117,1.082931,3.680554,3.834922,4.102912,5.174089,nan
1387,PL,Poland,"Drug dependence, toxicomania (F11-F16, F18-F19)",0.454545,0.818182,0.818182,0.150000,1.200000,1.650000,2.100000,1.350000,2.700000,3.750000,5.100000,nan
446,EE,Estonia,Other external causes of morbidity and mortality (remainder of V01-Y89),0.484848,0.393939,0.303030,1.000000,1.500000,0.500000,2.500000,4.500000,6.750000,5.750000,4.750000,nan
1382,PL,Poland,Dementia,0.454545,0.818182,0.818182,0.449421,1.130502,1.420077,1.315830,0.935907,0.866409,3.157529,4.686486,nan


### Causes of Death in countries with a high 2023 death ranking

index,abbr,name,cod,ddr2023_rank,vdr2023_rank,vdravg_rank,2015,2016,2017,2018,2019,2020,2021,2022,2023
223,CY,Cyprus,Other mental and behavioural disorders (remainder of F00-F99),0.969697,0.484848,0.515152,0.818182,1.363636,0.818182,2.454545,2.181818,4.636364,6.272727,7.363636,nan
183,CY,Cyprus,Chronic liver disease (excluding alcoholic and toxic liver disease),0.969697,0.484848,0.515152,nan,2.000000,0.000000,2.352941,2.882353,3.176471,2.941176,3.000000,nan
228,CY,Cyprus,Tuberculosis,0.969697,0.484848,0.515152,0.857143,0.857143,1.285714,1.714286,0.857143,0.857143,0.857143,2.142857,nan
185,CY,Cyprus,Chronic viral hepatitis B and C,0.969697,0.484848,0.515152,nan,2.000000,0.000000,0.400000,0.000000,2.000000,0.000000,2.000000,nan
195,CY,Cyprus,Human immunodeficiency virus [HIV] disease,0.969697,0.484848,0.515152,2.400000,0.600000,0.000000,0.000000,0.000000,1.200000,1.200000,2.400000,nan
186,CY,Cyprus,Dementia,0.969697,0.484848,0.515152,0.869388,0.942857,1.187755,1.530612,1.536735,1.346939,1.775510,1.983673,nan
193,CY,Cyprus,Falls,0.969697,0.484848,0.515152,0.634615,1.153846,1.211538,1.471154,1.528846,1.413462,1.875000,2.625000,nan
181,CY,Cyprus,Certain infectious and parasitic diseases (A00-B99),0.969697,0.484848,0.515152,1.002304,0.898618,1.099078,1.140553,1.389401,1.154378,1.389401,1.569124,nan
203,CY,Cyprus,"Malignant neoplasm of lip, oral cavity, pharynx",0.969697,0.484848,0.515152,1.062500,1.250000,0.687500,1.187500,1.437500,1.687500,1.437500,1.625000,nan
215,CY,Cyprus,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",0.969697,0.484848,0.515152,0.945378,0.932773,1.121849,1.008403,1.210084,1.285714,1.613445,1.764706,nan


In [20]:
# print some random ones with the colors

abbrs = cod.abbr.drop_duplicates().to_list()
random.shuffle(abbrs)

cods = cod.cod.drop_duplicates().to_list()
random.shuffle(cods)

for a in abbrs[0:5]:
    for c in cods[0:1]:
        try:
            temp = cod[(cod.abbr == a) & (cod.cod == c) ]
            temp = pd.pivot_table(
                data =  temp,
                values='deaths_norm',
                index = ['abbr','name','cod'],
                columns='year',
                aggfunc='mean'
            )
            temp.columns.name = 'index' 
            temp = temp.reset_index()
            display(temp.style.background_gradient(cmap=heatmapCM,axis=1))
        except Exception as e:
            pass
            print(f"Error: {e} | a={a} | c={c}")


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
0,SE,Sweden,Hodgkin disease and lymphomas,0.967086,1.010649,1.022265,0.993224,1.022265,0.962730,0.897386,0.935140


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021
0,LV,Latvia,Hodgkin disease and lymphomas,0.921951,0.973171,1.104878,0.914634,1.104878,0.980488,1.024390


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
0,FR,France,Hodgkin disease and lymphomas,0.989609,1.000204,1.010187,0.954768,0.976976,0.997963,0.973716,0.995518


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,ES,Spain,Hodgkin disease and lymphomas,0.984947,0.983921,1.031132,1.001711,1.031817,1.034554,1.005816,0.988710,1.011632


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
0,HR,Croatia,Hodgkin disease and lymphomas,1.100200,0.982966,0.916834,0.925852,0.988978,0.946894,0.898798,0.850701


### notes
we don't have enough 2023 and 2024 data 

In [21]:

cod_counter = Counter()
name_counter = Counter()

temp = pd.pivot_table(
        data =  cod,
        values='deaths_norm',
        index = ['abbr','name','cod'],
        columns='year',
        aggfunc='mean'
    )
temp.columns.name = 'index' 
temp = temp.reset_index()
for y in [2024,2023,2022,2021,2020]:
    try:
        temp = temp.sort_values(by=y,ascending=False)
        print(f'sorted by {y}')
        break
    except:
        pass

top_causes = temp.cod.head(1000).to_list()
cod_counter.update(top_causes)

top_name = temp.name.head(1000).to_list()
name_counter.update(top_name)

display(temp.head(100).style.background_gradient(cmap=heatmapCM,axis=1))

sorted by 2023


c:\Users\JGarza\pythons\Python312\Lib\site-packages\pandas\io\formats\style.py:3809: RuntimeWarning: invalid value encountered in scalar subtract
  rng = smax - smin


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022,2023
1109,LI,Liechtenstein,Chronic liver disease (excluding alcoholic and toxic liver disease),nan,nan,nan,nan,inf,inf,inf,inf,inf
1111,LI,Liechtenstein,Dementia,nan,1.500000,0.500000,1.500000,0.500000,8.500000,6.000000,7.500000,9.000000
1122,LI,Liechtenstein,Ill-defined and unknown causes of mortality,1.200000,1.500000,0.300000,1.800000,2.400000,2.700000,1.200000,1.500000,5.700000
50,AT,Austria,Other mental and behavioural disorders (remainder of F00-F99),0.794118,1.014706,1.191176,1.191176,2.338235,2.250000,3.352941,3.441176,4.014706
1881,SK,Slovakia,Non-malignant neoplasms (benign and uncertain),0.886640,0.607287,1.506073,1.336032,2.064777,0.838057,1.748988,2.502024,3.595142
24,AT,Austria,Influenza (including swine flu),0.717557,0.557252,1.725191,3.480916,2.374046,2.305344,0.045802,2.633588,3.442748
341,CZ,Czechia,"Other symptoms, signs and abnormal clinical and laboratory findings (remainder of R00-R99)",0.904192,0.868263,1.227545,1.485030,2.365269,3.065868,3.113772,2.856287,3.293413
598,ES,Spain,Event of undetermined intent,1.071429,0.785714,1.142857,1.761905,2.190476,0.833333,1.452381,1.880952,3.095238
1120,LI,Liechtenstein,Hodgkin disease and lymphomas,nan,nan,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,3.000000
308,CZ,Czechia,Event of undetermined intent,0.882353,0.995475,1.122172,1.506787,1.927602,2.031674,2.683258,2.918552,2.678733


In [22]:
html_table = f"""
<h>Top 20 Causes of Death in the top 1000 causes of death (see above)</h>
<table>
    <tr>
        <th>Item</th>
        <th>Count</th>
        <th>bar</th>
    </tr>
    {''.join(f'<tr><td>{i[0]}</td><td>{i[1]}</td><td>{"#" * i[1]}</td></tr>' for i in cod_counter.most_common()[0:20])}
</table>
"""

display(HTML(html_table))

Item,Count,bar
Chronic liver disease (excluding alcoholic and toxic liver disease),18,##################
Dementia,18,##################
Event of undetermined intent,18,##################
Hodgkin disease and lymphomas,18,##################
Certain infectious and parasitic diseases (A00-B99),18,##################
"Drug dependence, toxicomania (F11-F16, F18-F19)",18,##################
Asthma and status asthmaticus,18,##################
Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism,18,##################
Diseases of kidney and ureter,18,##################
Falls,18,##################


In [23]:
ddnames = ddr2023.name.head(100).to_list()

html_table = f"""
<h>Top 20 Countries (abbr) in the top 1000 causes of death (see above)</h>
<table>
    <tr>
        <th>Item</th>
        <th>Count</th>
        <th>bar</th>
        <th>in ddnames list</th>
    </tr>
    {''.join(f'<tr><td>{i[0]}</td><td>{i[1]}</td><td>{"#" * i[1]}</td><td>{"Y" if i[0] in ddnames else "N"}</td></tr>' for i in name_counter.most_common(20))}
</table>
"""

display(HTML(html_table))

Item,Count,bar,in ddnames list
Austria,58,##########################################################,Y
Slovakia,58,##########################################################,Y
Czechia,58,##########################################################,Y
Spain,58,##########################################################,Y
Belgium,58,##########################################################,Y
Bulgaria,58,##########################################################,Y
Switzerland,58,##########################################################,Y
Cyprus,58,##########################################################,Y
Germany,58,##########################################################,Y
Denmark,58,##########################################################,Y


# lets combine it with the data from part 1

### top 10 causes of deaths for top 10 countries with the most deaths in 2023

In [24]:
abbrs = ddr2023.head(10).abbr.to_list()

counter = Counter()

print(abbrs)

for a in abbrs:
    try:
        temp = cod[(cod.abbr == a)]
        temp = pd.pivot_table(
            data =  temp,
            values='deaths_norm',
            index = ['abbr','name','cod'],
            columns='year',
            aggfunc='mean'
        )
        temp.columns.name = 'index' 
        temp = temp.reset_index()
        for y in [2024,2023,2022,2021,2020]:
            try:
                temp = temp.sort_values(by=y,ascending=False)
                print(f'sorted by {y}')
                break
            except:
                pass

        top_causes = temp.cod.head(20).to_list()
        counter.update(top_causes)
        
        display(temp.head(10).style.background_gradient(cmap=heatmapCM,axis=1))
    except Exception as e:
        pass
        print(f"Error: {e} | a={a} | c={c}")


cod20dd102023 = []
for i in counter.most_common()[0:20]:
    cod20dd102023.append(i[0])

html_table = f"""
<h>Top 20 Causes of Death in the countries with the top deaths in 2023</h>
<table>
    <tr>
        <th>Item</th>
        <th>Count</th>
        <th>bar</th>
    </tr>
    {''.join(f'<tr><td>{i[0]}</td><td>{i[1]}</td><td>{"#" * i[1]}</td></tr>' for i in counter.most_common()[0:20])}
</table>
"""

display(HTML(html_table))


['MT', 'CY', 'FI', 'IS', 'NL', 'IE', 'DE', 'DK', 'AT', 'LU']
sorted by 2021


c:\Users\JGarza\pythons\Python312\Lib\site-packages\pandas\io\formats\style.py:3809: RuntimeWarning: invalid value encountered in scalar subtract
  rng = smax - smin


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021
18,MT,Malta,Event of undetermined intent,nan,nan,nan,nan,inf,inf,inf
23,MT,Malta,Ill-defined and unknown causes of mortality,1.875000,0.375000,0.750000,3.750000,5.625000,4.125000,3.375000
43,MT,Malta,Other external causes of morbidity and mortality (remainder of V01-Y89),1.000000,1.000000,1.000000,3.000000,2.000000,5.000000,3.000000
45,MT,Malta,Other infectious and parasitic diseases (remainder of A00-B99),0.708333,1.041667,1.250000,2.000000,1.833333,2.708333,2.666667
42,MT,Malta,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",0.825000,0.900000,1.275000,0.975000,1.575000,1.500000,2.175000
7,MT,Malta,Certain infectious and parasitic diseases (A00-B99),0.750000,1.173913,1.076087,1.630435,1.597826,2.184783,2.152174
12,MT,Malta,Dementia,0.846316,0.966316,1.187368,1.610526,1.446316,2.040000,2.103158
50,MT,Malta,Other mental and behavioural disorders (remainder of F00-F99),2.000000,0.500000,0.500000,1.500000,1.500000,0.500000,2.000000
20,MT,Malta,Falls,0.788321,1.226277,0.985401,0.919708,1.138686,1.160584,1.686131
27,MT,Malta,Leukaemia,0.929577,0.676056,1.394366,1.140845,1.014085,1.521127,1.521127


sorted by 2022


c:\Users\JGarza\pythons\Python312\Lib\site-packages\pandas\io\formats\style.py:3809: RuntimeWarning: invalid value encountered in scalar subtract
  rng = smax - smin


...

sorted by 2023


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022,2023
50,AT,Austria,Other mental and behavioural disorders (remainder of F00-F99),0.794118,1.014706,1.191176,1.191176,2.338235,2.250000,3.352941,3.441176,4.014706
24,AT,Austria,Influenza (including swine flu),0.717557,0.557252,1.725191,3.480916,2.374046,2.305344,0.045802,2.633588,3.442748
1,AT,Austria,Accidental poisoning by and exposure to noxious substances,1.043478,0.847826,1.108696,0.815217,1.402174,1.108696,2.543478,2.576087,2.673913
23,AT,Austria,Ill-defined and unknown causes of mortality,1.178571,0.730519,1.090909,1.051948,1.081169,2.668831,2.386364,2.571429,2.610390
12,AT,Austria,Dementia,0.675152,0.902424,1.422424,1.613939,1.612727,1.490909,1.644848,1.780606,2.007273
15,AT,Austria,Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism,1.012678,0.950872,1.036450,1.364501,1.412044,0.998415,1.492868,1.673534,1.954041
9,AT,Austria,Chronic liver disease (excluding alcoholic and toxic liver disease),nan,2.000000,0.000000,1.847690,1.972534,1.940075,1.867665,1.987516,1.950062
43,AT,Austria,Other external causes of morbidity and mortality (remainder of V01-Y89),1.070876,0.838918,1.090206,1.171392,0.931701,1.055412,1.453608,1.623711,1.786082
45,AT,Austria,Other infectious and parasitic diseases (remainder of A00-B99),0.808273,1.044718,1.147010,1.287870,1.370039,1.205702,1.447177,1.351593,1.713807
39,AT,Austria,Other diseases of the musculoskeletal system and connective tissue (remainder of M00-M99),0.816406,1.089844,1.093750,1.238281,1.230469,1.125000,1.179688,1.152344,1.542969


sorted by 2022


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
4,LU,Luxembourg,Assault,1.500000,1.250000,0.250000,0.750000,1.000000,0.750000,1.250000,2.750000
23,LU,Luxembourg,Ill-defined and unknown causes of mortality,0.968750,0.968750,1.062500,1.031250,2.031250,2.218750,2.468750,2.625000
1,LU,Luxembourg,Accidental poisoning by and exposure to noxious substances,1.826087,0.391304,0.782609,1.304348,1.434783,2.347826,2.478261,2.086957
24,LU,Luxembourg,Influenza (including swine flu),1.846154,0.692308,0.461538,3.230769,2.307692,1.615385,0.000000,1.846154
45,LU,Luxembourg,Other infectious and parasitic diseases (remainder of A00-B99),0.895522,0.716418,1.388060,0.940299,0.940299,1.119403,1.074627,1.671642
38,LU,Luxembourg,Other diseases of the genitourinary system (remainder of N00-N99),0.937500,0.562500,1.500000,1.375000,1.312500,1.750000,1.625000,1.625000
7,LU,Luxembourg,Certain infectious and parasitic diseases (A00-B99),0.907895,0.697368,1.394737,0.894737,0.881579,1.092105,1.013158,1.552632
9,LU,Luxembourg,Chronic liver disease (excluding alcoholic and toxic liver disease),nan,2.000000,0.000000,1.902439,1.609756,1.902439,1.512195,1.512195
42,LU,Luxembourg,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",0.785235,0.926174,1.288591,0.825503,0.986577,0.966443,1.026846,1.449664
53,LU,Luxembourg,"Rheumatoid arthritis and arthrosis (M05-M06,M15-M19)",1.384615,0.923077,0.692308,0.923077,1.153846,0.923077,0.461538,1.384615


Item,Count,bar
Other mental and behavioural disorders (remainder of F00-F99),10,##########
"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",9,#########
Other diseases of the nervous system and the sense organs (remainder of G00-H95),9,#########
Ill-defined and unknown causes of mortality,8,########
Other diseases of the genitourinary system (remainder of N00-N99),8,########
Chronic liver disease (excluding alcoholic and toxic liver disease),8,########
Event of undetermined intent,7,#######
Dementia,7,#######
Accidental poisoning by and exposure to noxious substances,7,#######
Other diseases of the musculoskeletal system and connective tissue (remainder of M00-M99),7,#######


### top 10 causes of deaths for top 10 countries with the most vaccines in 2023

In [25]:
abbrs = vdr2023.head(10).abbr.to_list()

counter = Counter()

print(abbrs)

for a in abbrs:
    try:
        temp = cod[(cod.abbr == a)]
        temp = pd.pivot_table(
            data =  temp,
            values='deaths_norm',
            index = ['abbr','name','cod'],
            columns='year',
            aggfunc='mean'
        )
        temp.columns.name = 'index' 
        temp = temp.reset_index()
        for y in [2024,2023,2022,2021,2020]:
            try:
                temp = temp.sort_values(by=y,ascending=False)
                print(f'sorted by {y}')
                break
            except:
                pass

        top_causes = temp.cod.head(20).to_list()
        counter.update(top_causes)
        
        display(temp.head(10).style.background_gradient(cmap=heatmapCM,axis=1))
    except Exception as e:
        pass
        print(f"Error: {e} | a={a} | c={c}")


html_table = f"""
<h>Top 20 Causes of Death in the countries with the top vaccines in 2023</h>
<table>
    <tr>
        <th>Item</th>
        <th>Count</th>
        <th>bar</th>
        <th>in last list</th>
    </tr>
    {''.join(f'<tr><td>{i[0]}</td><td>{i[1]}</td><td>{"#" * i[1]}</td><td>{"Y" if i[0] in cod20dd102023 else "N"}</td></tr>' for i in counter.most_common(20))}
</table>
"""

display(HTML(html_table))

['FI', 'PT', 'IT', 'SE', 'FR', 'LT', 'PL', 'DK', 'IE', 'NL']
sorted by 2022


c:\Users\JGarza\pythons\Python312\Lib\site-packages\pandas\io\formats\style.py:3809: RuntimeWarning: invalid value encountered in scalar subtract
  rng = smax - smin


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
9,FI,Finland,Chronic liver disease (excluding alcoholic and toxic liver disease),nan,nan,nan,inf,inf,inf,inf,inf
11,FI,Finland,Chronic viral hepatitis B and C,nan,nan,nan,inf,inf,inf,inf,inf
23,FI,Finland,Ill-defined and unknown causes of mortality,1.464883,0.675585,0.859532,0.735786,0.682274,0.789298,1.535117,2.046823
17,FI,Finland,"Drug dependence, toxicomania (F11-F16, F18-F19)",0.579545,1.431818,0.988636,1.602273,1.363636,1.738636,1.840909,1.738636
36,FI,Finland,Other diseases of the circulatory system (remainder of I00-I99),0.949102,1.001744,1.049153,1.095639,1.214777,1.300975,1.339764,1.535557
42,FI,Finland,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",0.856354,1.088398,1.055249,1.110497,1.149171,1.215470,1.226519,1.497238
50,FI,Finland,Other mental and behavioural disorders (remainder of F00-F99),0.987578,1.024845,0.987578,1.229814,1.217391,1.198758,1.409938,1.490683
12,FI,Finland,Dementia,0.998815,1.005927,0.995258,1.101439,1.083658,1.144623,1.305165,1.429128
40,FI,Finland,Other diseases of the nervous system and the sense organs (remainder of G00-H95),0.967711,0.992403,1.039886,1.081671,1.171890,1.238367,1.246914,1.360874
13,FI,Finland,Diabetes mellitus,0.910006,1.021397,1.068597,1.017621,1.125236,1.261171,1.270610,1.346130


sorted by 2021


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021
17,PT,Portugal,"Drug dependence, toxicomania (F11-F16, F18-F19)",1.434783,0.391304,1.173913,1.043478,1.565217,1.695652,2.347826
50,PT,Portugal,Other mental and behavioural disorders (remainder of F00-F99),0.917197,1.136943,0.945860,1.337580,2.025478,2.149682,2.207006
1,PT,Portugal,Accidental poisoning by and exposure to noxious substances,0.864629,0.917031,1.218341,1.401747,1.414847,1.296943,2.148472
23,PT,Portugal,Ill-defined and unknown causes of mortality,1.019554,0.976008,1.004439,1.153791,1.454295,1.755518,1.835053
12,PT,Portugal,Dementia,0.887734,1.004329,1.107937,1.337374,1.544589,1.751804,1.662915
35,PT,Portugal,"Other accidents (W20-W64, W75-X39, X50-X59, Y86)",0.769384,1.017325,1.213292,1.176654,0.605794,1.011360,1.583073
45,PT,Portugal,Other infectious and parasitic diseases (remainder of A00-B99),0.930394,0.995789,1.073817,1.050037,0.995046,1.028486,1.405251
42,PT,Portugal,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",1.068902,0.974587,0.956510,0.989521,0.877129,1.107414,1.393503
2,PT,Portugal,"Accidents (V01-X59, Y85, Y86)",0.892690,0.983984,1.123327,1.093811,0.892690,1.057087,1.302483
43,PT,Portugal,Other external causes of morbidity and mortality (remainder of V01-Y89),1.018310,1.073239,0.908451,0.954930,1.212676,1.276056,1.297183


...

index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
17,IE,Ireland,"Drug dependence, toxicomania (F11-F16, F18-F19)",2.000000,nan,0.000000,2.000000,0.000000,10.000000,8.000000,32.000000
23,IE,Ireland,Ill-defined and unknown causes of mortality,1.325581,0.732558,0.941860,1.186047,1.534884,2.825581,4.430233,4.186047
43,IE,Ireland,Other external causes of morbidity and mortality (remainder of V01-Y89),1.000000,1.857143,0.142857,0.714286,1.571429,1.142857,1.571429,4.142857
35,IE,Ireland,"Other accidents (W20-W64, W75-X39, X50-X59, Y86)",0.915789,0.934737,1.149474,1.471579,1.768421,2.065263,2.349474,2.482105
9,IE,Ireland,Chronic liver disease (excluding alcoholic and toxic liver disease),1.445344,1.554656,0.000000,2.040486,2.016194,1.991903,2.574899,2.477733
51,IE,Ireland,"Other symptoms, signs and abnormal clinical and laboratory findings (remainder of R00-R99)",0.829493,0.981567,1.188940,1.147465,1.534562,1.935484,1.811060,2.211982
50,IE,Ireland,Other mental and behavioural disorders (remainder of F00-F99),0.914286,1.000000,1.085714,1.285714,1.200000,1.028571,1.771429,1.942857
39,IE,Ireland,Other diseases of the musculoskeletal system and connective tissue (remainder of M00-M99),0.906178,0.851259,1.242563,1.187643,1.112128,1.324943,1.263158,1.709382
42,IE,Ireland,"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",0.982487,0.992995,1.024518,1.329247,1.171629,1.444834,1.528897,1.670753
38,IE,Ireland,Other diseases of the genitourinary system (remainder of N00-N99),0.936842,1.052632,1.010526,1.063158,1.126316,1.321053,1.210526,1.478947


sorted by 2022


c:\Users\JGarza\pythons\Python312\Lib\site-packages\pandas\io\formats\style.py:3809: RuntimeWarning: invalid value encountered in scalar subtract
  rng = smax - smin


index,abbr,name,cod,2015,2016,2017,2018,2019,2020,2021,2022
9,NL,Netherlands,Chronic liver disease (excluding alcoholic and toxic liver disease),nan,nan,nan,inf,inf,inf,inf,inf
11,NL,Netherlands,Chronic viral hepatitis B and C,nan,nan,nan,inf,inf,inf,inf,inf
23,NL,Netherlands,Ill-defined and unknown causes of mortality,0.793319,1.035491,1.171190,1.172582,1.299930,3.081420,3.815588,4.358386
17,NL,Netherlands,"Drug dependence, toxicomania (F11-F16, F18-F19)",0.828358,1.119403,1.052239,0.962687,1.432836,1.500000,1.835821,2.037313
18,NL,Netherlands,Event of undetermined intent,1.236641,0.801527,0.961832,0.870229,1.145038,1.007634,1.259542,1.763359
20,NL,Netherlands,Falls,0.887891,1.032518,1.079591,1.264168,1.294518,1.432951,1.470115,1.686900
24,NL,Netherlands,Influenza (including swine flu),0.928808,0.851821,1.219371,3.009934,1.343543,0.735099,0.052152,1.504967
2,NL,Netherlands,"Accidents (V01-X59, Y85, Y86)",0.932499,1.017589,1.049913,1.189866,1.195981,1.255562,1.283867,1.498078
19,NL,Netherlands,External causes of morbidity and mortality (V01-Y89),0.949282,1.007179,1.043539,1.131461,1.134375,1.177449,1.205828,1.371537
1,NL,Netherlands,Accidental poisoning by and exposure to noxious substances,0.861751,1.013825,1.124424,1.059908,1.105991,1.225806,1.253456,1.350230


Item,Count,bar,in last list
"Other endocrine, nutritional and metabolic diseases (remainder of E00-E90)",10,##########,Y
Chronic liver disease (excluding alcoholic and toxic liver disease),9,#########,Y
"Drug dependence, toxicomania (F11-F16, F18-F19)",9,#########,N
Other mental and behavioural disorders (remainder of F00-F99),9,#########,Y
Other diseases of the nervous system and the sense organs (remainder of G00-H95),9,#########,Y
Ill-defined and unknown causes of mortality,8,########,Y
Other diseases of the genitourinary system (remainder of N00-N99),8,########,Y
Other diseases of the musculoskeletal system and connective tissue (remainder of M00-M99),7,#######,Y
"Other accidents (W20-W64, W75-X39, X50-X59, Y86)",7,#######,N
Other infectious and parasitic diseases (remainder of A00-B99),7,#######,Y
